In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torch.utils.data import random_split
from torchvision.datasets import DatasetFolder

In [2]:
torch.cuda.set_device(0)
device = torch.device("cuda")
torch.cuda.is_available()

True

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

In [4]:
trainset = ImageFolder("train/", transform=transform)

# Określenie proporcji podziału (np. 80% danych treningowych, 20% danych testowych)
train_size = int(0.8 * len(trainset))
test_size = len(trainset) - train_size

# Podział trainset na zbiory treningowy i testowy
train_dataset, test_dataset = random_split(trainset, [train_size, test_size])

# Utworzenie DataLoader dla każdego zbioru danych
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                            shuffle=True, num_workers=16)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=False, num_workers=16)


# test_dataset = ImageFolder('test_all', transform=transform)

# test_all_set = DatasetFolder("test_all/", loader=lambda x: Image.open(x), extensions="jpg", transform=transform)


# test_all_loader = torch.utils.data.DataLoader(test_all_set, batch_size=batch_size,
#                                          shuffle=False, num_workers=16)


classes = trainset.classes

/home/natalia/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        ## Warstwa max pooling 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Poprawka na max pooling
        self.fc1 = nn.Linear(16 * 13 * 13, 120)  # 16 * 13 * 13 zamiast 16 * 5 * 5
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)
# net

In [6]:


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [7]:
images, labels = next(iter(train_loader))
images[0:1].size()

torch.Size([1, 3, 64, 64])

In [8]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('[%d/5] loss: %.3f' %
          (epoch+1 ,  running_loss / 2000))
    running_loss = 0.0

print('Finished Training')

[1/5] loss: 3.375
[2/5] loss: 2.856
[3/5] loss: 2.611
[4/5] loss: 2.451
[5/5] loss: 2.330
Finished Training


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        # calculate outputs by running images through the network 
        outputs = net(images).cpu()
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 34 %


In [10]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data    
        images = images.to(device)
        outputs = net(images).cpu()   
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy for class acoustic is: 16.6 %
Accuracy for class antenna is: 39.0 %
Accuracy for class bacteria is: 34.6 %
Accuracy for class battery is: 29.3 %
Accuracy for class bean  is: 40.0 %
Accuracy for class beetle is: 54.6 %
Accuracy for class bicycle is: 40.5 %
Accuracy for class birch is: 13.7 %
Accuracy for class bird  is: 9.3 %
Accuracy for class bomb  is: 35.0 %
Accuracy for class bread is: 23.9 %
Accuracy for class bridge is: 36.5 %
Accuracy for class camera is: 33.6 %
Accuracy for class carbon is: 0.0 %
Accuracy for class cat   is: 22.7 %
Accuracy for class corn  is: 11.9 %
Accuracy for class crab  is: 14.6 %
Accuracy for class crocodilian is: 31.1 %
Accuracy for class echinoderm is: 20.5 %
Accuracy for class egg   is: 20.9 %
Accuracy for class elephant is: 22.5 %
Accuracy for class fish  is: 44.8 %
Accuracy for class flower is: 82.4 %
Accuracy for class frog  is: 19.1 %
Accuracy for class fungus is: 59.7 %
Accuracy for class gauge is: 42.7 %
Accuracy for class hammer is: 69.2